In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 6개, 3x3의 정사각 컨볼루션 행렬
        # 컨볼루션 커널 정의
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 아핀(affine) 연산: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6은 이미지 차원에 해당
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # (2, 2) 크기 윈도우에 대해 맥스 풀링(max pooling)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 크기가 제곱수라면 하나의 숫자만을 특정
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 배치 차원을 제외한 모든 차원
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [12]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [13]:
input = torch.randn(1,1,32,32)
print(input)
out = net(input)
print(out)

tensor([[[[ 0.7696, -1.0921,  0.2014,  ...,  2.5634,  0.1310,  2.3673],
          [-1.2165,  0.2384, -0.7092,  ...,  0.8775,  0.3828,  0.5686],
          [-1.4171,  2.2775,  0.7893,  ..., -0.0583, -0.6556,  1.2724],
          ...,
          [ 0.6666,  0.1137,  0.0913,  ...,  0.7542,  2.3879, -0.3786],
          [ 0.6596,  0.8911, -1.0940,  ...,  0.2341,  0.1369, -0.0233],
          [ 2.7417,  0.6517,  1.0069,  ..., -0.7783,  0.6558,  1.7329]]]])
tensor([[-0.1193,  0.1022, -0.1310, -0.0541, -0.0769, -0.1644, -0.0070,  0.0844,
          0.1382,  0.0290]], grad_fn=<AddmmBackward>)


In [14]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [15]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.3674, grad_fn=<MseLossBackward>)


In [16]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

loss.backward()를 실행할때, 전체 그래프는 loss에 대해 미분되며, 그래프 내의 requires_grad=True인 모든 Tensor는 gradient가 누적된 .grad tensor를 갖게됨.